In [1]:
# 모듈 로딩
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import urllib.request
import random
import os
import cv2

import tensorflow as tf
from keras.models import Sequential, Model, save_model, load_model
from keras.applications import vgg16
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical, plot_model, set_random_seed

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

plt.rcParams['font.family'] = 'Malgun Gothic'
# warnings.filterwarnings(action='ignore'

In [2]:
# 랜덤값 고정
np.random.seed(42)
tf.random.set_seed(42)
set_random_seed(42)

In [3]:
# train, test 데이터 불러오기
train=pd.read_csv('/MainData/train.csv')
test=pd.read_csv('/MainData/test.csv')

### [1] 데이터 살펴보기

In [4]:
train.head()

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식


In [5]:
train.isnull().sum(), test.isnull().sum()

(id          0
 img_path    0
 overview    0
 cat1        0
 cat2        0
 cat3        0
 dtype: int64,
 id          0
 img_path    0
 overview    0
 dtype: int64)

In [6]:
train.duplicated().sum(), test.duplicated().sum()

(0, 0)

In [7]:
print(train.cat1.value_counts(), '\n')
print(train.cat2.value_counts(), '\n')
print(train.cat3.value_counts())

인문(문화/예술/역사)    5614
음식              4912
레포츠             2611
자연              1719
숙박              1434
쇼핑               696
Name: cat1, dtype: int64 

음식점       4912
육상 레포츠    2288
자연관광지     1578
역사관광지     1571
숙박시설      1434
문화시설      1220
휴양관광지      889
체험관광지      805
쇼핑         696
축제         569
건축/조형물     328
수상 레포츠     277
관광자원       141
공연/행사      139
산업관광지       93
복합 레포츠      24
항공 레포츠      11
레포츠소개       11
Name: cat2, dtype: int64 

한식                3438
야영장,오토캠핑장         1558
바/까페               785
유적지/사적지            622
일반축제               549
                  ... 
MTB                  2
대중콘서트                2
인라인(실내 인라인 포함)       2
스카이다이빙               2
클래식음악회               2
Name: cat3, Length: 128, dtype: int64


In [8]:
# 카테고리 대, 중, 소별 갯수
len(train.cat1.value_counts()), len(train.cat2.value_counts()), len(train.cat3.value_counts())

(6, 18, 128)

## [1-2] 이미지 불러오기

In [9]:
IMG_PATH='/MainData/image/train/'
img_path_list=os.listdir(IMG_PATH)

In [21]:
img_list=[]
file_list=[]

for file in img_path_list:
    im = plt.imread(IMG_PATH + file)
    im = cv2.resize(im, (300, 200))
    im = im.reshape(-1, 300, 200, 3)
    img_list.append(im)
    file_list.append(file.split('.jpg')[0])

In [22]:
img_list[0].shape, img_list[:2]

((1, 300, 200, 3),
 [array([[[[ 80,  87, 105],
           [ 79,  87, 105],
           [ 81,  88, 106],
           ...,
           [ 50,  73, 107],
           [ 49,  72, 106],
           [ 49,  71, 108]],
  
          [[ 49,  71, 108],
           [ 49,  71, 108],
           [ 49,  71, 108],
           ...,
           [ 75,  91, 117],
           [ 74,  90, 116],
           [ 75,  91, 117]],
  
          [[ 75,  91, 117],
           [ 76,  92, 118],
           [ 75,  91, 117],
           ...,
           [ 37,  56,  91],
           [ 41,  57,  91],
           [ 41,  57,  91]],
  
          ...,
  
          [[105,  89,  76],
           [ 85,  71,  58],
           [ 80,  67,  54],
           ...,
           [111,  98,  81],
           [100,  87,  71],
           [109,  98,  80]],
  
          [[118, 107,  89],
           [112,  99,  82],
           [108,  94,  78],
           ...,
           [ 97,  85,  69],
           [ 92,  83,  65],
           [102,  92,  76]],
  
          [[ 96,  86,  

In [23]:
file_list[:5]

['TRAIN_00000', 'TRAIN_00001', 'TRAIN_00002', 'TRAIN_00003', 'TRAIN_00004']

In [24]:
img_arr=np.concatenate(img_list)

In [31]:
# img_arr[0]

## [2] 전처리

In [44]:
# 라벨 인코더
le = LabelEncoder()
target1 = le.fit_transform(train.cat1)
target2 = le.fit_transform(train.cat2)
target3 = le.fit_transform(train.cat3)

In [45]:
target1.shape, target2.shape, target3.shape

((16986,), (16986,), (16986,))

In [46]:
# train, test데이터 분리
X_train, X_test, y_train, y_test=train_test_split(img_arr, target2,
                                                 stratify=target2,
                                                 random_state=42,
                                                 test_size=0.2)

In [47]:
X_train[0], X_train.shape, y_train[0], y_train.shape

(array([[[229, 230, 234],
         [229, 230, 234],
         [229, 230, 234],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [254, 254, 254],
         [254, 254, 254],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [235, 245, 255],
         [235, 245, 255],
         [234, 244, 254]],
 
        ...,
 
        [[ 97,  84,  73],
         [109,  93,  78],
         [ 84,  68,  52],
         ...,
         [146, 128, 110],
         [146, 127, 110],
         [135, 114,  96]],
 
        [[142, 121, 102],
         [152, 131, 114],
         [136, 117, 100],
         ...,
         [108, 102,  85],
         [115, 100,  84],
         [107,  92,  74]],
 
        [[133, 115,  97],
         [ 98,  82,  66],
         [106,  95,  80],
         ...,
         [103,  87,  70],
  

## [3] 모델 구상

In [116]:
# 모델 구성
model = Sequential()

model.add(Conv2D(32, kernel_size=(3), # 자동으로 (3,3)
                 activation='relu',
                 input_shape=(300, 200, 3))) # (28, 28), 1은 흑백
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # 축소
model.add(Dropout(0.25)) # 과적합 방지
model.add(Flatten()) # 펼침
model.add(Dense(128, activation='relu')) # 펼치고 Dense하면 정확도 상승
model.add(Dropout(0.5)) # 과적합 방지
model.add(Dense(18, activation='softmax')) # 출력층

In [117]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 298, 198, 32)      896       
                                                                 
 conv2d_29 (Conv2D)          (None, 296, 196, 64)      18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 148, 98, 64)      0         
 g2D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 148, 98, 64)       0         
                                                                 
 flatten_14 (Flatten)        (None, 928256)            0         
                                                                 
 dense_28 (Dense)            (None, 128)               118816896 
                                                     

In [118]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## [3-2] 모델 생성

In [119]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics='accuracy')

In [120]:
MODEL_PATH='/MainData/'
stopCB = EarlyStopping(monitor='val_loss', patience=5)
cpCB = ModelCheckpoint(filepath=MODEL_PATH, verbose=1, save_best_only=True)

## [3-3] 모델 학습

In [121]:
history=model.fit(X_train, y_train,
                 epochs=20,
                 batch_size=64,
                 validation_data=(X_test, y_test),
                 callbacks=[stopCB, cpCB])

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 231.2430 - accuracy: 0.2503
Epoch 1: val_loss improved from inf to 2.75878, saving model to /MainData\


PermissionDeniedError: {{function_node __wrapped__MergeV2Checkpoints_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to create a directory: /; Permission denied [Op:MergeV2Checkpoints]

In [ ]:
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(20, 8))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend(fontsize=10)
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()
plot_accuracy_and_loss(history)